# Limpeza e Análise de Dados

O objetivo desta atividade consiste na limpeza e análise dos dados obtidos na raspagem realizada na aba de despesas do site da Sefaz. Para isto, procederemos como segue: 
0. Preparação
1. Limpeza dos dados
2. Estatísticas e Visualização
3. Análise e Discussão

Estes passos podem ocorrer de maneira cíclica até que o agente esteja satisfeito.

## 0. Preparação

### Importação de Bibliotecas

In [ ]:
%matplotlib inline 
import pandas as pd # manipulação de dados
import matplotlib.pyplot as plt # visualização
import seaborn as sns #visualização
sns.set(style="whitegrid") # Pra ficar bonitinho

### Leitura do conjunto de dados

In [ ]:
df_fonte = pd.read_csv('output/sefaz-Fonte de Recurso.csv', index_col=0)
print(df_fonte.shape)
df_fonte

## 1. Limpeza

Esta etapa envolve a remoção de colunas e linhas incompletas, irrelevantes ou erradas.

### Remoção

df_fonte.drop(itens, axis=?, inplace=?)

itens: colunas ou linhas que devem ser removidas

axis: 
* 0: linhas
* 1: colunas

inplace: transcrever a base atual com a nova? (True ou False)

In [ ]:
df_fonte.drop(['0', '1'], axis=1, inplace=True)

df_fonte

#### Aumentar a quantidade de linhas que podem ser visualizadas:

In [ ]:
pd.options.display.max_rows = 10000

#### Visualizar o conjunto de dados novamente

In [ ]:
print(df_fonte.shape)
df_fonte

#### Remover NaNs

df_fonte.dropna(subset=?, inplace=?)

subset: conjunto de linhas ou colunas que devem ser analisadas

In [ ]:
df_fonte.dropna(subset=['Fonte de Recurso', 'Dotação Inicial', 
                        'Autorizado', 'Empenhado', 'Liquidado', 
                        'Pago', 'Pago Exercício Anterior', 
                        'A Pagar Exercício Anterior'], inplace=True)
print(df_fonte.shape)

#### Problema: O cabeçalho se repete!

Isolando linhas em que isso ocorre:

In [ ]:
df_fonte.loc[df_fonte['Dotação Inicial']=='Dotação Inicial']

#### Removendo linhas em que o cabeçalho se repete:

In [ ]:
df_fonte.drop(df_fonte.loc[df_fonte['Dotação Inicial']=='Dotação Inicial'].index, inplace=True)
df_fonte

#### Verificar totais

In [ ]:
df_fonte.loc[df_fonte['Fonte de Recurso']=='Total:']

#### Remover linhas duplicadas

Note que há repetições em que a Dotação Inicil está descrita com .00, e outras em que está com .0 !!

Os dados estão em formato de texto! Vamos convertê-los para reais:

In [ ]:
df_fonte.dtypes

In [ ]:
df_fonte[['Dotação Inicial',
          'Autorizado', 'Empenhado', 'Liquidado', 
          'Pago', 'Pago Exercício Anterior', 
          'A Pagar Exercício Anterior']] = df_fonte[['Dotação Inicial',
                                                     'Autorizado', 'Empenhado', 'Liquidado', 
                                                     'Pago', 'Pago Exercício Anterior', 
                                                     'A Pagar Exercício Anterior']].astype(float)

df_fonte.drop_duplicates(keep='first', inplace=True)
df_fonte.shape

#### Verificar totais novamente: 

In [ ]:
df_fonte.loc[df_fonte['Fonte de Recurso']=='Total:']

#### Atribuir totais a outro conjunto de dados e removê-los do conjunto principal

In [ ]:
df_fonte_totais = df_fonte.loc[df_fonte['Fonte de Recurso']=='Total:']
df_fonte.drop(df_fonte.loc[df_fonte['Fonte de Recurso']=='Total:'].index, inplace=True)

In [ ]:
df_fonte_totais

In [ ]:
df_fonte

### Dados limpos!!!

## 2. Estatísticas e Visualização

Para ajudar na visualização: 

In [ ]:
pd.options.display.float_format = '{:20,.2f}'.format

#### Descrição dos conjuntos:

In [ ]:
df_fonte.describe()

In [ ]:
df_fonte_totais.describe()

Plotar quantidade de vezes que cada fonte de recurso esteve presente nos relatórios da Sefaz

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(8,18))
sns.countplot(ax=ax, y='Fonte de Recurso', data=df_fonte)
ax.set_xlabel("Quantidade")
ax.figure.savefig("output/graficos/linha-total.png")

Plotar um gráfico de barras que mostre os valores a ser pagos do exercício anterior por ano

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(10,4))
sns.barplot(ax=ax, x='ano', y='A Pagar Exercício Anterior', data=df_fonte_totais)
ax.set_xlabel("Ano")
ax.figure.savefig("output/graficos/barra-a-pagar.png")

Plotar um gráfico de barras que mostre os valores a ser pagos do exercício anterior por fonte de recurso

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(10, 30))
sns.barplot(ax=ax, x='A Pagar Exercício Anterior', y='Fonte de Recurso', data=df_fonte)
ax.figure.savefig("output/graficos/barra-a-pagar.png")

Plotar um gráfico de linhas com os valores em cada ano

In [ ]:
df_fonte_totais.columns.drop(['ano', 'periodo', 'consulta'])

In [ ]:
df_fonte_totais.index = df_fonte_totais['ano']
df_fonte_totais.drop(['ano', 'grupo', 'periodo', 'consulta','Fonte de Recurso'], axis=1, inplace=True)

In [ ]:
plt.clf()
fig, ax = plt.subplots(figsize=(20, 10))
sns.lineplot(data=df_fonte_totais[['Dotação Inicial','Autorizado','Empenhado',
                                   'Liquidado','Pago', 'Pago Exercício Anterior', 
                                   'A Pagar Exercício Anterior']], dashes=False)
ax.figure.savefig("output/graficos/linha-total.png")